In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [100]:
transaction_df = pd.read_csv("./transaction_data.csv")

In [44]:
transaction_df.shape

(2595732, 12)

In [101]:
hh_demographic_df = pd.read_csv('hh_demographic_version1.csv')

In [28]:
hh_demographic_df

,age,income,marital_status,household_size,household_key
0,65+,35-49K,Married,2,1
1,45-54,50-74K,Married,2,7
2,25-34,25-34K,Single,3,8
3,25-34,75-99K,Single,4,13
4,45-54,50-74K,Unknown,1,16
...,...,...,...,...,...
796,35-44,50-74K,Single,2,2494
797,45-54,75-99K,Married,3,2496
798,45-54,35-49K,Single,1,2497
799,25-34,50-74K,Single,2,2498


### 在交易資料中新增會員及非會員標籤

In [102]:
data = pd.merge(transaction_df,hh_demographic_df,how='left',on='household_key')

In [103]:
data = data.drop(['income','marital_status','household_size'],axis=1)

In [46]:
data

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,age,income,marital_status,household_size
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,NaN,NaN,NaN,NaN
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,NaN,NaN,NaN,NaN
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,NaN,NaN,NaN,NaN
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,NaN,NaN,NaN,NaN
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595727,1598,42305362535,711,92130,1,0.99,3228,0.00,1520,102,0.0,0.0,NaN,NaN,NaN,NaN
2595728,1598,42305362535,711,114102,1,8.89,3228,0.00,1520,102,0.0,0.0,NaN,NaN,NaN,NaN
2595729,1598,42305362535,711,133449,1,6.99,3228,0.00,1520,102,0.0,0.0,NaN,NaN,NaN,NaN
2595730,1598,42305362535,711,6923644,1,4.50,3228,-0.49,1520,102,0.0,0.0,NaN,NaN,NaN,NaN


In [42]:
type(data.iloc[0,-1])

float

In [104]:
data['age'] = data['age'].replace(np.nan,'非會員')

In [105]:
data['age'] = data['age'].apply(lambda x : "會員" if x != '非會員' else x)

In [106]:
data = data.rename(columns={'age': 'member_status'})

### 發現部分不是會員的人也享有會員折扣

In [107]:
data

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,member_status
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,非會員
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,非會員
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,非會員
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,非會員
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,非會員
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595727,1598,42305362535,711,92130,1,0.99,3228,0.00,1520,102,0.0,0.0,非會員
2595728,1598,42305362535,711,114102,1,8.89,3228,0.00,1520,102,0.0,0.0,非會員
2595729,1598,42305362535,711,133449,1,6.99,3228,0.00,1520,102,0.0,0.0,非會員
2595730,1598,42305362535,711,6923644,1,4.50,3228,-0.49,1520,102,0.0,0.0,非會員


In [108]:
# Yes = 有折扣, No = 沒折扣
data['RETAIL_status'] = data['RETAIL_DISC'].apply(lambda x : 'Yes' if x <0 else 'No')

In [109]:
data['RETAIL_status'].value_counts()

Yes    1303026
No     1292706
Name: RETAIL_status, dtype: int64

In [110]:
data['member_status'].value_counts()

會員     1427303
非會員    1168429
Name: member_status, dtype: int64

### 將交易時間轉換成小時

### 時間格式又分成 單一小時 (1-24)，小時分鐘(1212 or 312)

In [163]:
data.query('TRANS_TIME== 3')

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,member_status,RETAIL_status
95139,1406,27900947425,78,1123434,1,2.17,327,-0.22,3,12,0.0,0.0,非會員,Yes
101945,1770,27936868246,81,860776,1,0.59,432,0.00,3,12,0.0,0.0,非會員,No
101946,1770,27936868246,81,868522,1,6.99,432,0.00,3,12,0.0,0.0,非會員,No
101947,1770,27936868246,81,879948,1,0.25,432,-0.35,3,12,0.0,0.0,非會員,Yes
101948,1770,27936868246,81,931396,1,4.59,432,0.00,3,12,0.0,0.0,非會員,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2548645,1086,42136037984,700,965543,1,1.09,404,0.00,3,101,0.0,0.0,非會員,No
2548646,1086,42136037984,700,1014509,3,7.50,404,-0.57,3,101,0.0,0.0,非會員,Yes
2548647,1086,42136037984,700,1135476,2,6.98,404,0.00,3,101,0.0,0.0,非會員,No
2548648,1086,42136037984,700,1138508,1,0.79,404,-0.10,3,101,0.0,0.0,非會員,Yes


In [26]:
transaction_df

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC,member
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0,會員
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0,非會員
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0,會員
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0,非會員
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0,會員
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595727,1598,42305362535,711,92130,1,0.99,3228,0.00,1520,102,0.0,0.0,非會員
2595728,1598,42305362535,711,114102,1,8.89,3228,0.00,1520,102,0.0,0.0,非會員
2595729,1598,42305362535,711,133449,1,6.99,3228,0.00,1520,102,0.0,0.0,非會員
2595730,1598,42305362535,711,6923644,1,4.50,3228,-0.49,1520,102,0.0,0.0,會員


In [3]:
transaction_df.shape

(2595732, 12)

In [7]:
transaction_df.head()

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [18]:
transaction_df[transaction_df['COUPON_DISC'] != 0]

,household_key,BASKET_ID,DAY,PRODUCT_ID,QUANTITY,SALES_VALUE,STORE_ID,RETAIL_DISC,TRANS_TIME,WEEK_NO,COUPON_DISC,COUPON_MATCH_DISC
129,718,26985360571,1,830503,1,2.99,324,-1.00,1115,1,-1.00,0.00
134,718,26985360571,1,855325,1,1.10,324,-0.69,1115,1,-0.40,-0.40
151,718,26985360571,1,934676,1,1.37,324,-0.42,1115,1,-1.00,0.00
154,718,26985360571,1,948756,3,2.12,324,-0.90,1115,1,-0.75,-0.25
160,718,26985360571,1,965292,2,3.50,324,-1.28,1115,1,-1.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
2595361,1823,42289906117,711,1134790,1,3.39,402,0.00,1334,102,-0.70,-0.30
2595363,1823,42289906117,711,6979803,1,3.69,402,0.00,1334,102,-0.50,-0.50
2595369,1823,42289906117,711,17249766,1,2.56,402,0.00,1334,102,-1.00,0.00
2595589,303,42289918892,711,5978656,0,0.00,343,0.00,1513,102,-1.00,0.00


### 已兌換折價券的資料

In [8]:
coupon_redempt_df = pd.read_csv("./coupon_redempt.csv")

In [32]:
coupon_redempt_df.sort_values(by="CAMPAIGN")

,household_key,DAY,COUPON_UPC,CAMPAIGN
139,93,375,51380040009,1
1105,1299,361,57218050076,2
1104,1299,361,53800030050,2
1103,1299,361,51530011350,2
1102,1299,361,51111074175,2
...,...,...,...,...
892,972,360,57756765875,30
2062,2331,343,51111073935,30
2063,2331,343,51111074635,30
1387,1634,365,53700012075,30


In [29]:
for i in range(coupon_redempt_df.shape[0]):
    print(coupon_redempt_df.iloc[i,:])
    print('------------------------------------------')

household_key              1
DAY                      421
COUPON_UPC       10000085364
CAMPAIGN                   8
Name: 0, dtype: int64
------------------------------------------
household_key              1
DAY                      421
COUPON_UPC       51700010076
CAMPAIGN                   8
Name: 1, dtype: int64
------------------------------------------
household_key              1
DAY                      427
COUPON_UPC       54200000033
CAMPAIGN                   8
Name: 2, dtype: int64
------------------------------------------
household_key              1
DAY                      597
COUPON_UPC       10000085476
CAMPAIGN                  18
Name: 3, dtype: int64
------------------------------------------
household_key              1
DAY                      597
COUPON_UPC       54200029176
CAMPAIGN                  18
Name: 4, dtype: int64
------------------------------------------
household_key              8
DAY                      422
COUPON_UPC       53600000078
CAMPAIGN

Name: 421, dtype: int64
------------------------------------------
household_key            389
DAY                      596
COUPON_UPC       10000085476
CAMPAIGN                  18
Name: 422, dtype: int64
------------------------------------------
household_key            389
DAY                      609
COUPON_UPC       52000030042
CAMPAIGN                  18
Name: 423, dtype: int64
------------------------------------------
household_key            389
DAY                      609
COUPON_UPC       52113100033
CAMPAIGN                  18
Name: 424, dtype: int64
------------------------------------------
household_key            389
DAY                      609
COUPON_UPC       54060060076
CAMPAIGN                  18
Name: 425, dtype: int64
------------------------------------------
household_key            389
DAY                      609
COUPON_UPC       54300026050
CAMPAIGN                  18
Name: 426, dtype: int64
------------------------------------------
household_key     

------------------------------------------
household_key            911
DAY                      544
COUPON_UPC       10000085429
CAMPAIGN                  13
Name: 837, dtype: int64
------------------------------------------
household_key            911
DAY                      593
COUPON_UPC       10000085475
CAMPAIGN                  18
Name: 838, dtype: int64
------------------------------------------
household_key            920
DAY                      507
COUPON_UPC       10000085425
CAMPAIGN                  13
Name: 839, dtype: int64
------------------------------------------
household_key            920
DAY                      507
COUPON_UPC       10000085429
CAMPAIGN                  13
Name: 840, dtype: int64
------------------------------------------
household_key            920
DAY                      507
COUPON_UPC       51600070033
CAMPAIGN                  13
Name: 841, dtype: int64
------------------------------------------
household_key            920
DAY          

------------------------------------------
household_key           1475
DAY                      523
COUPON_UPC       52740022050
CAMPAIGN                  13
Name: 1243, dtype: int64
------------------------------------------
household_key           1475
DAY                      593
COUPON_UPC       10000085479
CAMPAIGN                  18
Name: 1244, dtype: int64
------------------------------------------
household_key           1475
DAY                      593
COUPON_UPC       10000089252
CAMPAIGN                  18
Name: 1245, dtype: int64
------------------------------------------
household_key           1475
DAY                      593
COUPON_UPC       51800015050
CAMPAIGN                  18
Name: 1246, dtype: int64
------------------------------------------
household_key           1475
DAY                      593
COUPON_UPC       51809461078
CAMPAIGN                  18
Name: 1247, dtype: int64
------------------------------------------
household_key           1475
DAY     

Name: 1662, dtype: int64
------------------------------------------
household_key           1937
DAY                      655
COUPON_UPC       55100000033
CAMPAIGN                  23
Name: 1663, dtype: int64
------------------------------------------
household_key           1944
DAY                      435
COUPON_UPC       10000085364
CAMPAIGN                   8
Name: 1664, dtype: int64
------------------------------------------
household_key           1944
DAY                      435
COUPON_UPC       10000089044
CAMPAIGN                   8
Name: 1665, dtype: int64
------------------------------------------
household_key           1944
DAY                      435
COUPON_UPC       51862700076
CAMPAIGN                   8
Name: 1666, dtype: int64
------------------------------------------
household_key           1944
DAY                      435
COUPON_UPC       52370020076
CAMPAIGN                   8
Name: 1667, dtype: int64
------------------------------------------
household_ke

Name: 2076, dtype: int64
------------------------------------------
household_key           2351
DAY                      521
COUPON_UPC       10000089134
CAMPAIGN                  13
Name: 2077, dtype: int64
------------------------------------------
household_key           2351
DAY                      549
COUPON_UPC       55340000082
CAMPAIGN                  13
Name: 2078, dtype: int64
------------------------------------------
household_key           2351
DAY                      598
COUPON_UPC       10000085475
CAMPAIGN                  18
Name: 2079, dtype: int64
------------------------------------------
household_key           2351
DAY                      598
COUPON_UPC       53120026033
CAMPAIGN                  18
Name: 2080, dtype: int64
------------------------------------------
household_key           2351
DAY                      605
COUPON_UPC       55400061078
CAMPAIGN                  18
Name: 2081, dtype: int64
------------------------------------------
household_ke

### 有收到特定活動資訊的客人

In [4]:
campaign_table_df = pd.read_csv("./campaign_table.csv")

In [16]:
campaign_table_df.household_key.unique()

array([  17,   27,  212, ..., 2386,  498,  369], dtype=int64)

In [22]:
campaign_table_df.groupby('CAMPAIGN').count()

,DESCRIPTION,household_key
CAMPAIGN,,
1,13,13
2,48,48
3,12,12
4,81,81
5,166,166
6,65,65
7,198,198
8,1076,1076
9,176,176


### 有收到活動資訊，且有使用優惠券的人

In [33]:
dataset=campaign_table_df.merge(coupon_redempt_df,on=['household_key','CAMPAIGN'])

In [37]:
dataset

,DESCRIPTION,household_key,CAMPAIGN,DAY,COUPON_UPC
0,TypeA,2443,26,228,57457010076
1,TypeA,2305,26,237,57143030076
2,TypeA,2305,26,237,57756765876
3,TypeA,2305,26,260,54900020032
4,TypeA,2280,26,225,57008510075
...,...,...,...,...,...
2313,TypeC,256,20,638,54589399275
2314,TypeC,19,20,684,54100027032
2315,TypeC,1917,6,396,53663271277
2316,TypeC,1017,15,575,55000035213


### 活動起訖日

In [14]:
campaign_desc_df = pd.read_csv("./campaign_desc.csv")

In [24]:
campaign_desc_df.sort_values(by=['START_DAY','CAMPAIGN'],ascending=True)

,DESCRIPTION,CAMPAIGN,START_DAY,END_DAY
29,TypeA,26,224,264
28,TypeC,27,237,300
27,TypeB,28,259,320
26,TypeB,29,281,334
25,TypeA,30,323,369
23,TypeB,1,346,383
24,TypeB,2,351,383
20,TypeC,3,356,412
22,TypeB,4,372,404
21,TypeB,5,377,411


### 活動中有那些商品有折價

In [ ]:
coupon_df = pd.read_csv("./coupon.csv")

In [20]:
coupon_df.head()

,COUPON_UPC,PRODUCT_ID,CAMPAIGN
0,10000089061,27160,4
1,10000089064,27754,9
2,10000089073,28897,12
3,51800009050,28919,28
4,52100000076,28929,25
